## Make sure to change all paths as they are all local

In [ ]:
import importlib
import Embeddings
importlib.reload(Embeddings)
from Embeddings import Embedders_Five
import numpy as np
from sklearn.preprocessing import LabelEncoder
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import urllib.request
import os
import zipfile
import gensim
import gensim.downloader as api
import fasttext
import fasttext.util
# download fastText
fastext_path = "/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/cc.en.300.bin"

if not os.path.exists(fastext_path):
    fasttext.util.download_model('en', if_exists='ignore')  # English


#Download Word2Vec model
word2vec_path = "/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/word2vec-google-news-300.bin"

if not os.path.exists(word2vec_path):
    word2vec_model = api.load('word2vec-google-news-300')
    word2vec_model.save_word2vec_format('word2vec-google-news-300.bin', binary=True)

# Download GloVe vectors
glove_url = "https://nlp.stanford.edu/data/glove.6B.zip"
zip_path = "/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/glove.6B.zip"
glove_txt = "/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/glove.6B.100d.txt"
glove_word2vec = "/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/glove.6B.100d.word2vec"

if not os.path.exists(zip_path):
    print("Downloading GloVe vectors...")
    urllib.request.urlretrieve(glove_url, zip_path)
    
    # Unzip the file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/')
    print("Download and extraction complete")
    
    # Convert to Word2Vec format
    print("Converting to Word2Vec format...")
    from gensim.scripts.glove2word2vec import glove2word2vec
    glove2word2vec(glove_txt, glove_word2vec)
    print("Conversion complete")
else:
    if not os.path.exists(glove_word2vec):
        print("Converting to Word2Vec format...")
        from gensim.scripts.glove2word2vec import glove2word2vec
        glove2word2vec(glove_txt, glove_word2vec)
        print("Conversion complete")
    print("Files already exist")



# Load data
user_stories = pd.read_excel("/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/Dataset/Domain_Classification_Data/Synthetic User Stories.xlsx")
user_stories['Domain'] = user_stories['Domain'].str.lower()

# Create embedder instance
embedder = Embedders_Five(user_stories["User Story"])

# Encode labels
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(user_stories["Domain"])
print("Number of labels:", data_y.shape)
domains_names = np.unique(user_stories["Domain"])
print("Unique domains:", domains_names)

# Domains Test

In [ ]:
#-----------------------------------------------------------------
# Test FastText
print("\n=== FastText Results ===")
fasttext_features = embedder.getFastTextEmbedding()
# Split data for FastText
X_train_fasttext, X_test_fasttext, y_train_fasttext, y_test_fasttext = train_test_split(
    fasttext_features, data_y, test_size=0.2, random_state=42
)
# LazyClassifier for FastText
clf_fasttext = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_fasttext, predictions_fasttext = clf_fasttext.fit(X_train_fasttext, X_test_fasttext, y_train_fasttext, y_test_fasttext)
print("\nFastText Models Performance:")
print(models_fasttext)

#-----------------------------------------------------------------
# Test GloVe
print("\n=== GloVe Results ===")
glove_features = embedder.getGloVEEmbedding()
# Split data for GloVe
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(
    glove_features, data_y, test_size=0.2, random_state=42
)
# LazyClassifier for GloVe
clf_glove = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_glove, predictions_glove = clf_glove.fit(X_train_glove, X_test_glove, y_train_glove, y_test_glove)
print("\nGloVe Models Performance:")
print(models_glove)

#-----------------------------------------------------------------
# Test TFIDF
print("\n=== TFIDF Results ===")
data_x = embedder.getTFIDFEmbeddings()
# Split data for TFIDF
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)
X_train_dense = X_train
X_test_dense = X_test
# LazyClassifier for TFIDF
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_tfidf, predictions_tfidf = clf.fit(X_train_dense, X_test_dense, y_train, y_test)
print("\nTFIDF Models Performance:")
print(models_tfidf)

#-----------------------------------------------------------------
# Test BERT tokenization approach
print("\n=== BERT Results ===")
bert_features = embedder.getBERTEmbeddings()
# Convert to float for ML compatibility if needed
bert_features = bert_features.astype(np.float32)
# Split data for BERT
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(
    bert_features, data_y, test_size=0.2, random_state=42
)
# LazyClassifier for BERT
clf_bert = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_bert, predictions_bert = clf_bert.fit(X_train_bert, X_test_bert, y_train_bert, y_test_bert)
print("\nBERT Models Performance:")
print(models_bert)

#-----------------------------------------------------------------
# Test Word2Vec
print("\n=== Word2Vec Results ===")
w2v_features = embedder.getWord2VecEmbedding()
# Split data for Word2Vec
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
    w2v_features, data_y, test_size=0.2, random_state=42
)
# LazyClassifier for Word2Vec
clf_w2v = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_w2v, predictions_w2v = clf_w2v.fit(X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v)
print("\nWord2Vec Models Performance:")
print(models_w2v)

# Compare best models
print("\n=== Performance Comparison ===")
print("Best FastText Model:", models_fasttext.iloc[0])
print("Best TFIDF Model:", models_tfidf.iloc[0])
print("Best BERT Model:", models_bert.iloc[0])
print("Best Word2Vec Model:", models_w2v.iloc[0])
print("Best GloVe Model:", models_glove.iloc[0])

# Sensitive Features test

In [13]:
from SensitiveFeaturesMapping import SensitiveFeaturesMapper
import importlib
import Embeddings
importlib.reload(Embeddings)
from Embeddings import Embedders_Five
import numpy as np
from sklearn.preprocessing import LabelEncoder
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import urllib.request
import os
import zipfile
import gensim.downloader as api
import fasttext
import fasttext.util


# Load data
user_stories = pd.read_excel("/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/Dataset/Domain_Classification_Data/Synthetic User Stories.xlsx")
user_stories['Domain'] = user_stories['Domain'].str.lower()
ontology = SensitiveFeaturesMapper("/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/Dataset/Feature_Extraction/domains-features-mapping.csv",
                                   "/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/Dataset/Feature_Extraction/tasks-features-mapping.csv")


labels = pd.read_excel("/Users/ahmed/Desktop/CanWeTrustReFAIR/CanWeTrustReFAIR/Dataset/ML_Tasks_Classification_Data/Keyword labelled.xlsx", header=None)
labels[2] = labels[2].apply(lambda x: x.lower())
categories_column = []
for row in labels.iterrows():
    current_labels = []
    for label in row[1][3:]:
        if isinstance(label, str):
            current_labels.append(label.lower())
    categories_column.append(current_labels)
labels["Categories array"] = categories_column
labels[[2, "Categories array"]]

target = []
counter = 0
for row in user_stories.iterrows():
    target.append(labels[labels[2]==row[1]["Machine Learning Task"].lower()]["Categories array"].values[0])
    counter += 1
user_stories["Target"] = target
user_stories[["User Story","Target"]]

ontology.get_sensitive_features(user_stories["Target"][10000],user_stories["Domain"][10000])


                                              User Story  \
0      A group of researchers is using abstractive su...   
1      As a plant scientist, I want to use abstractiv...   
2      As a molecular biologist, I want to use action...   
3      As a plant scientist, I want to use action mod...   
4      As a bioinformatics researcher, I want to use ...   
...                                                  ...   
12396  As a computer vision researcher, I want to use...   
12397  As a network engineer, I want to use word2vec ...   
12398  As a computer vision researcher, I want to use...   
12399  As a network engineer, I want to use WordNet t...   
12400  As a computer vision researcher, I want to use...   

                                                  Target  
0                                   [data summarization]  
1                                   [data summarization]  
2                                                [other]  
3                                          

## Test Embeddings

In [ ]:
import importlib
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Reload custom modules
import Embeddings
import ML_Classification
importlib.reload(Embeddings)
importlib.reload(ML_Classification)
from Embeddings import Embedders_Five
from ML_Classification import ML_Classification

# Load and preprocess data
# Make sure to change the path, copy path of dataset -> domain_classification_data
user_stories = pd.read_excel("/Users/Claudia/Projects/CanWeTrustReFAIR_/CanWeTrustReFAIR/Dataset/Domain_Classification_Data/Synthetic User Stories.xlsx")
user_stories['Domain'] = user_stories['Domain'].str.lower()

# Create embedder instance
embedder = Embedders_Five(user_stories["User Story"])

# Encode labels
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(user_stories["Domain"])

# Print dataset information
print("Number of labels:", data_y.shape)
domains_names = np.unique(user_stories["Domain"])
print("Unique domains:", domains_names)

# Generate TFIDF embeddings
print("\n=== TFIDF Results ===")
data_x = embedder.getTFIDFEmbeddings()

# Split data for TFIDF
X_train, X_test, y_train, y_test = train_test_split(
    data_x, data_y, 
    test_size=0.2, 
    random_state=42
)

classifier = ML_Classification()
best_model, performance_df = classifier.train_ml_models(X_train, y_train, X_test, y_test)

print("Best Model:", best_model)
print("\nAll Models Performance:") 
print(performance_df)

# Test MoJo

In [74]:
import pandas as pd 
import pickle
from Embeddings import Embedders_Five
data = pd.read_excel("./Dataset/SensitiveFeaturesValidation_Data/oracle-predictions.xlsx")
domain_task_mapping = pd.read_csv("./Dataset/domains-tasks-mapping.csv")
domains_mapping = pd.read_csv("./Dataset/Feature_Extraction/domains-features-mapping.csv")
tasks_mapping = pd.read_csv("./Dataset/Feature_Extraction/tasks-features-mapping.csv")
data.head(15)


,User Story,Oracle Domain,Oracle Tasks & Sensitive Features,Oracle Unique Sensitive Features
0,A bioinformatics company is using statistical ...,Biology,"{'classification': [], 'graph mining': []}",{}
1,A bioinformatics company is using text generat...,Biology,{'graph mining': []},{}
2,A bioinformatics company is using visual quest...,Biology,{'classification': []},{}
3,A bioinformatics researcher is developing a sy...,Biology,{'graph mining': []},{}
4,A bioinformatics researcher is interested in i...,Biology,{},{}
5,A bioinformatics researcher is using text to s...,Biology,{},{}
6,A bioinformatics researcher is working on a pr...,Biology,{},{}
7,A bioinformatics team is using hierarchical cl...,Biology,{'graph mining': []},{}
8,A bioinformatics team is using learning under ...,Biology,{'classification': []},{}
9,A bioinformatics team is using medoids to iden...,Biology,{},{}


In [75]:
# Load models
with open('./MLModels/xgboost_best_model.pkl', 'rb') as f:
   xgb_model = pickle.load(f)
with open('./MLModels/linear_svc_final_model.pkl', 'rb') as f:
   svc_model = pickle.load(f)
    
# Embed Data
embedder = Embedders_Five(data["User Story"])
bert_samples = embedder.getBERTEmbeddings()
glove_samples = embedder.getGloVEEmbedding()


## ReFAIR prediction and export the new excel

In [76]:
# Use the exact same order as during training
model_domains = np.array(['Biology', 'Plant Science', 'Economics', 'Finance & Marketing',
       'Information Systems', 'News', 'Law', 'Political Science', 'Library',
       'Linguistics', 'Literature', 'Cardiology', 'Dermatology', 'Endocrinology',
       'Health', 'Medicine', 'Nephrology', 'Pediatrics', 'Pharmacology', 'Psychology',
       'Radiology', 'Psycology', 'Demography', 'Education', 'Social Media',
       'Social Networks', 'Social Work', 'Sociology', 'Transportation',
       'Urban Studies', 'Demograpy', 'Social networks', 'Social work', 'Movies',
       'Music', 'Sport', 'Computer Networks', 'Computer Vision'])

def standardize_domain(predicted_idx):
    """Maps the predicted index to standardized domain name"""
    predicted_domain = model_domains[predicted_idx].lower()
    
    # Fix known typos
    if predicted_domain == 'psycology':
        return 'psychology'
    elif predicted_domain == 'demograpy':
        return 'demography'
    elif predicted_domain == 'social networks':  # Handle duplicate with different case
        return 'social networks'
    elif predicted_domain == 'social work':      # Handle duplicate with different case
        return 'social work'
    
    return predicted_domain

# Process predictions
refair_predictions = []

for i in range(len(data)):
    user_story = data.iloc[i]['User Story']
    
    # Get standardized domain prediction
    predicted_domain = standardize_domain(domain_predictions[i])
    
    # Rest of your prediction code...
    task_vector = task_predictions[i].toarray()[0]
    predicted_tasks = [mltasks_classes[j] for j, val in enumerate(task_vector) if val == 1]
    
    # Format tasks dictionary
    tasks_features_dict = {}
    if predicted_tasks:
        for task in predicted_tasks:
            # Check if task is valid for predicted domain
            valid_task = domain_task_mapping[
                (domain_task_mapping['Domain'].str.lower() == predicted_domain.lower()) & 
                (domain_task_mapping['Task'].str.lower() == task.lower())
            ].shape[0] > 0
            
            if valid_task:
                tasks_features_dict[task] = []
    
    # Get sensitive features
    if tasks_features_dict:
        refair_tasks_features = mapper.get_sensitive_features(list(tasks_features_dict.keys()), predicted_domain)
        refair_unique_features = mapper.get_unique_sensitive_features(list(tasks_features_dict.keys()), predicted_domain)
    else:
        refair_tasks_features = {}
        refair_unique_features = set()
    
    refair_row = {
        'User Story': user_story,
        'ReFAIR Domain': predicted_domain,
        'ReFAIR Tasks & Sensitive Features': str(refair_tasks_features),
        'ReFAIR Unique Sensitive Features': str(refair_unique_features)
    }
    refair_predictions.append(refair_row)

# Convert to DataFrame
refair_df = pd.DataFrame(refair_predictions)
final_df = pd.concat([data, refair_df[['ReFAIR Domain', 'ReFAIR Tasks & Sensitive Features', 'ReFAIR Unique Sensitive Features']]], axis=1)

final_df.head(100)

,User Story,Oracle Domain,Oracle Tasks & Sensitive Features,Oracle Unique Sensitive Features,ReFAIR Domain,ReFAIR Tasks & Sensitive Features,ReFAIR Unique Sensitive Features
0,A bioinformatics company is using statistical ...,Biology,"{'classification': [], 'graph mining': []}",{},biology,{},set()
1,A bioinformatics company is using text generat...,Biology,{'graph mining': []},{},biology,{},set()
2,A bioinformatics company is using visual quest...,Biology,{'classification': []},{},biology,{'classification': []},set()
3,A bioinformatics researcher is developing a sy...,Biology,{'graph mining': []},{},biology,{'graph mining': []},set()
4,A bioinformatics researcher is interested in i...,Biology,{},{},biology,{},set()
...,...,...,...,...,...,...,...
95,A literary researcher is using machine learnin...,Literature,"{'clustering': ['author', 'gender', 'textual r...","{'author', 'gender', 'textual references to pe...",literature,{'clustering': ['textual references to people ...,{'textual references to people and their demog...
96,A literary scholar wants to develop a bidirect...,Literature,{'ranking': ['gender']},{'gender'},literature,{},set()
97,A literary scholar wants to use automated patt...,Literature,"{'ranking': ['gender'], 'clustering': ['author...","{'author', 'gender', 'textual references to pe...",literature,{'clustering': ['textual references to people ...,{'textual references to people and their demog...
98,A literary scholar wants to use bootstrap aggr...,Literature,{},{},literature,{},set()


In [79]:
for col in final_df.select_dtypes(include='object').columns:
    final_df[col] = final_df[col].str.lower()

# Define the output file path
output_file_path = '/home/ahmedyra/projects/def-hinat/ahmedyra/EECS6448Project/CanWeTrustReFAIR/Dataset/SensitiveFeaturesValidation_Data/overall-oracle-prediciton.csv'

# Export the DataFrame to CSV
final_df.to_csv(output_file_path, index=False)

print(f"CSV file saved to {output_file_path}")

CSV file saved to /home/ahmedyra/projects/def-hinat/ahmedyra/EECS6448Project/CanWeTrustReFAIR/Dataset/SensitiveFeaturesValidation_Data/overall-oracle-prediciton.csv
